In [7]:
import numpy as np
def render_event_voxels(event_stream):
    T, H, W = event_stream.shape
    img_outs = []
    img_0_inds = []
    img_pos_inds = []
    img_neg_inds = []
    for t in range(T):
        img_acc = event_stream[t]
        img_out = np.full((H,W,3), fill_value=255,dtype='uint8')

        # Simple thresholding
        #img_out[img_acc > 0] = [0,0,255]
        #img_out[img_acc < 0] = [255,0,0]

        # With weighting (more complicated alternative)
        clip_percentile = 80
        min_percentile = -np.percentile(np.abs(img_acc[img_acc < 0]), clip_percentile)
        max_percentile = np.percentile(np.abs(img_acc[img_acc > 0]), clip_percentile)
        img_acc = np.clip(img_acc, min_percentile, max_percentile)


        img_acc_max = img_acc.max()
        idx_pos = img_acc > 0
        img_acc[idx_pos] = img_acc[idx_pos]/img_acc_max
        val_pos = img_acc[idx_pos]
        img_out[idx_pos] = np.stack((255-val_pos*255, 255-val_pos*255, np.ones_like(val_pos)*255), axis=1)

        img_acc_min = img_acc.min()
        idx_neg = img_acc < 0
        img_acc[idx_neg] = img_acc[idx_neg]/img_acc_min
        val_neg = img_acc[idx_neg]
        img_out[idx_neg] = np.stack((np.ones_like(val_neg)*255, 255-val_neg*255, 255-val_neg*255), axis=1)
        img_outs.append(img_out)

        idx_zero = img_acc == 0

        img_0_inds.append(idx_zero)
        img_pos_inds.append(idx_pos)
        img_neg_inds.append(idx_neg)
    return img_outs, np.array(img_0_inds), np.array(img_pos_inds), np.array(img_neg_inds)

In [8]:
def cal_single_iou(pred, gt):
    keep = gt != 255
    hist = np.bincount(gt[keep] * 11 + pred[keep], minlength=11**2).reshape(11,11)
    ious = np.diag(hist) / (hist.sum(0) + hist.sum(1) - np.diag(hist))
    # ious[np.isnan(ious)] = 0.
    miou = ious.mean()
    ious *= 100
    miou *= 100
    return ious, miou

In [10]:
import numpy as np
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm

semseg_dict = {
    "num_classes": 11,
    "ignore_label": 255,
    "class_names": [
        "background", "building", "fence", "person", "pole",
        "road", "sidewalk", "vegetation", "car", "wall",
        "traffic sign",
    ],
    "color_map": np.array([
        [0, 0, 0], [70, 70, 70], [190, 153, 153], [220, 20, 60], [153, 153, 153], 
        [128, 64, 128], [244, 35, 232], [107, 142, 35], [0, 0, 142], [102, 102, 156], 
        [220, 220, 0],
    ]),
}
# scenes = ['day', 'night']
scenes = ['day']
# splits = ['train', 'val']
splits = ['val']
rgb_day_dir = Path('/home/xiaoshan/work/adap_v/DELIVER/output/DSEC_CMNeXt-B2_i/dsec_rgb_day_eval_20240927_120256')
rgbe_day_dir = Path('/home/xiaoshan/work/adap_v/DELIVER/output/DSEC_CMNeXt-B2_ie/dsec_rgbe_day_eval_20240927_120221') 

results = {}
results['rgb'] = {}
results['rgbe'] = {}
for scene in scenes:
    for split in splits:
        data_path = Path(f'/home/xiaoshan/work/adap_v/my_proj/data/DSEC/{scene}/')
        event_03_dir = data_path / 'event_03' / split
        img_dir = data_path / 'leftImg8bit' / split
        lbl_dir = data_path / 'gtFine' / split
        
        seqs = sorted(list(event_03_dir.glob('*')))
        for seq in seqs:
            seq = str(seq.relative_to(event_03_dir))
            print("Processing sequence: ", seq)
            seq_event_03_dir = data_path / 'event_03' / split / seq

            for event_03_file in tqdm(sorted(seq_event_03_dir.glob('*.npy'))):
                event_03_file_str = str(event_03_file)
                img_file = Path(event_03_file_str.replace('event_03', 'leftImg8bit').replace('.npy', '.png'))
                lbl_file = Path(event_03_file_str.replace('event_03', 'gtFine').replace('.npy', '_gtFine_labelTrainIds11.png'))
                rgb_day_file_id = rgb_day_dir / seq / str(event_03_file.relative_to(seq_event_03_dir)).replace('.npy', '.png')
                rgb_day_file_color = rgb_day_dir / seq / str(event_03_file.relative_to(seq_event_03_dir)).replace('.npy', '_labelTrainIds11.png')
                rgbe_day_file_id = rgbe_day_dir / seq / str(event_03_file.relative_to(seq_event_03_dir)).replace('.npy', '.png')
                rgbe_day_file_color = rgbe_day_dir / seq / str(event_03_file.relative_to(seq_event_03_dir)).replace('.npy', '_labelTrainIds11.png')

                event_03 = np.load(event_03_file)[:, :440, :]
                # event_03 = np.transpose(event_03, (1, 2, 0))  # 使用 numpy.transpose 替代 permute
                rendered_events, img_0_inds, img_pos_inds, img_neg_inds = render_event_voxels(event_03) 
                img = Image.open(img_file)
                lbl_id = Image.open(lbl_file)
                lbl_color = semseg_dict['color_map'][np.array(lbl_id)]
                rgb_day_id = Image.open(rgb_day_file_id)
                rgb_day_color = Image.open(rgb_day_file_color)
                rgbe_day_id = Image.open(rgbe_day_file_id)
                rgbe_day_color = Image.open(rgbe_day_file_color)

                rgb_iou, rgb_miou = cal_single_iou(np.array(rgb_day_id), np.array(lbl_id))
                rgbe_iou, rgbe_miou = cal_single_iou(np.array(rgbe_day_id), np.array(lbl_id))
                # print(f'RGB: {rgb_miou:.2f}, RGBE: {rgbe_miou:.2f}')
                # print(f'RGB: {rgb_iou}, RGBE: {rgbe_iou}')
                results['rgb'][f'{seq}_{event_03_file.stem}'] = {}
                results['rgbe'][f'{seq}_{event_03_file.stem}'] = {}
                results['rgb'][f'{seq}_{event_03_file.stem}']['miou'] = rgb_miou
                results['rgbe'][f'{seq}_{event_03_file.stem}']['miou'] = rgbe_miou
                results['rgb'][f'{seq}_{event_03_file.stem}']['iou'] = rgb_iou
                results['rgbe'][f'{seq}_{event_03_file.stem}']['iou'] = rgbe_iou

    #             # 如果任意一个的 miou 小于 50，可视化
    #             if rgb_miou > 50 and rgbe_miou > 50:
    #                 continue
    #             print('Visualizing...')

    #             lbl_color = Image.fromarray(lbl_color.astype(np.uint8))
                
    #             # 可视化事件图像
    #             fig, axes = plt.subplots(3, 3, figsize=(15, 15))

    #             # 第一行
    #             axes[0, 0].imshow(rendered_events[-1])
    #             axes[0, 0].set_title('Rendered Events')
    #             axes[0, 1].imshow(img)
    #             axes[0, 1].set_title('Image')
    #             axes[0, 2].imshow(lbl_color)
    #             axes[0, 2].set_title('Label')

    #             # 第二行
    #             axes[1, 0].imshow(img)
    #             axes[1, 0].imshow(rendered_events[-1], alpha=0.7)
    #             axes[1, 0].set_title('Overlay Image and Events')
    #             axes[1, 1].imshow(img)
    #             axes[1, 1].imshow(lbl_color, alpha=0.7)
    #             axes[1, 1].set_title('Overlay Image and Label')
    #             axes[1, 2].imshow(img)
    #             axes[1, 2].imshow(rendered_events[-1], alpha=0.7)
    #             axes[1, 2].imshow(lbl_color, alpha=0.7)
    #             axes[1, 2].set_title('Overlay Image, Events and Label')

    #             # 第三行
    #             axes[2, 0].imshow(rgb_day_color)
    #             axes[2, 0].set_title('RGB Day Color')
    #             axes[2, 1].imshow(rgbe_day_color)
    #             axes[2, 1].set_title('RGBE Day Color')
    #             axes[2, 2].imshow(rgbe_day_color, alpha=0.7)
    #             axes[2, 2].imshow(rendered_events[-1], alpha=0.5)
    #             axes[2, 2].set_title('Overlay Events and RGBE Day Color')
    #             # tight layout
    #             plt.tight_layout()
    #             plt.savefig(f'./visualize/{seq}_{event_03_file.stem}.png')

    #             break
    #         break
    #     break
    # break

Processing sequence:  zurich_city_13_a


 21%|██▏       | 9/42 [00:00<00:02, 14.09it/s]/tmp/ipykernel_1225835/1448291431.py:4: RuntimeWarning: invalid value encountered in true_divide
  ious = np.diag(hist) / (hist.sum(0) + hist.sum(1) - np.diag(hist))
100%|██████████| 42/42 [00:02<00:00, 16.88it/s]


Processing sequence:  zurich_city_14_c


100%|██████████| 133/133 [00:07<00:00, 18.58it/s]


Processing sequence:  zurich_city_15_a


100%|██████████| 138/138 [00:08<00:00, 16.87it/s]


In [17]:
nan_classes = []
# 遍历 results['rgb'] 中的每个项目
for class_name, metrics in results['rgb'].items():
    # 检查是否有 NaN
    if np.isnan(metrics['miou']):
        for key, value in metrics.items():
            print(f"RGB: {class_name} {key}: {value}")
        nan_classes.append(class_name)

# 输出所有出现过 NaN 的类别
print("出现过 NaN 的类别有:", nan_classes)

类别 zurich_city_13_a_000101 出现过 NaN
类别 zurich_city_13_a_000110 出现过 NaN
类别 zurich_city_13_a_000155 出现过 NaN
类别 zurich_city_13_a_000182 出现过 NaN
类别 zurich_city_13_a_000218 出现过 NaN
类别 zurich_city_13_a_000227 出现过 NaN
类别 zurich_city_13_a_000236 出现过 NaN
类别 zurich_city_13_a_000245 出现过 NaN
类别 zurich_city_13_a_000263 出现过 NaN
类别 zurich_city_13_a_000272 出现过 NaN
类别 zurich_city_13_a_000290 出现过 NaN
类别 zurich_city_13_a_000299 出现过 NaN
类别 zurich_city_13_a_000308 出现过 NaN
类别 zurich_city_13_a_000326 出现过 NaN
类别 zurich_city_14_c_000164 出现过 NaN
类别 zurich_city_14_c_000173 出现过 NaN
类别 zurich_city_14_c_000200 出现过 NaN
类别 zurich_city_14_c_000218 出现过 NaN
类别 zurich_city_14_c_000380 出现过 NaN
类别 zurich_city_14_c_000479 出现过 NaN
类别 zurich_city_14_c_000542 出现过 NaN
类别 zurich_city_14_c_000551 出现过 NaN
类别 zurich_city_14_c_000560 出现过 NaN
类别 zurich_city_14_c_000569 出现过 NaN
类别 zurich_city_14_c_000578 出现过 NaN
类别 zurich_city_14_c_000614 出现过 NaN
类别 zurich_city_14_c_000632 出现过 NaN
类别 zurich_city_14_c_000641 出现过 NaN
类别 zurich_city_14_c_